In [168]:
import requests
import json
import os
import re
import time
import string
import pandas as pd
from googlesearch import search
from bs4 import BeautifulSoup as bs
from joblib import Parallel, delayed

In [2]:
request_timeout = 10
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Cache-Control': 'private, max-age=0',
}
html_pattern = "<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>"

#search variables
from_year = 2013
to_year = 2022
total_news_per_year = 100

In [3]:
"""
def get_google_news_urls(q, page, from_year, to_year):
    urls = []
    queries_tag = "Gx5Zad fP1Qef xpd EtOod pkphOe"
    title_tag = "BNeawe vvjwJb AP7Wnd"
    search_url = f'https://www.google.com/search?q={q}&lr=lang_en&cd_min=1/1/2015&cd_max=2013&tbs=sbd:1,lr:lang_en,cdr:1,cd_min:1/1/2015,cd_max:2015&tbm=nws&start={page * 10}'
    print(search_url)
    search_result = requests.get(search_url)
    soup = bs(search_result.content, "html.parser")
    body = soup.find(id="main")
    queries = body.find_all("div", queries_tag)
    for i in queries:
        output = {}
        header = i.find('a')
        title = header.find('div', 'BNeawe vvjwJb AP7Wnd').text
        url = header['href']
        url = url.replace('/url?q=', '')
        url = url[0: s.index("&sa")]
        output['url'] = url
        output['title'] = title
        urls.append(output)    
        
    return urls
"""

'\ndef get_google_news_urls(q, page, from_year, to_year):\n    urls = []\n    queries_tag = "Gx5Zad fP1Qef xpd EtOod pkphOe"\n    title_tag = "BNeawe vvjwJb AP7Wnd"\n    search_url = f\'https://www.google.com/search?q={q}&lr=lang_en&cd_min=1/1/2015&cd_max=2013&tbs=sbd:1,lr:lang_en,cdr:1,cd_min:1/1/2015,cd_max:2015&tbm=nws&start={page * 10}\'\n    print(search_url)\n    search_result = requests.get(search_url)\n    soup = bs(search_result.content, "html.parser")\n    body = soup.find(id="main")\n    queries = body.find_all("div", queries_tag)\n    for i in queries:\n        output = {}\n        header = i.find(\'a\')\n        title = header.find(\'div\', \'BNeawe vvjwJb AP7Wnd\').text\n        url = header[\'href\']\n        url = url.replace(\'/url?q=\', \'\')\n        url = url[0: s.index("&sa")]\n        output[\'url\'] = url\n        output[\'title\'] = title\n        urls.append(output)    \n        \n    return urls\n'

In [4]:
def get_google_news_urls(q, total, from_year, to_year):
    urls = []
    for url in search(q, tld="co.in", start=0, stop=total, num=20, pause=2, lang='en', tbs=f'sbd:1,lr:lang_en,cdr:1,cd_min:1/1/{from_year},cd_max:{to_year}', extra_params={'tbm': 'nws'}):
        urls.append(url)
    return urls

In [5]:
def get_news_data(url):
    output = {}
    text = ""
    title = ""
    datetime = ""
    try:
        article = requests.get(url, headers=headers, timeout=request_timeout)
        status_code = article.status_code
        if status_code >= 400:
            f = open('ERROR_LOG.txt', 'a')
            f.write(f'[{time.asctime(time.localtime())}] Code {status_code}: {url}\n')
            f.close()
            return None
        soup = bs(article.content, "html.parser")
        datetime = soup.find('meta', property='article:published_time')
        paragraphs = soup.find_all("p")
        if soup.find('title') is not None:
            title = soup.find('title').text
        if datetime is not None:
            datetime = datetime['content']
        if paragraphs is not None:
            for p in paragraphs:
                text += re.sub(html_pattern, '', p.text).strip() + ' '  
        output['title'] = title
        output['text'] = text
        output['datetime'] = datetime
        output['url'] = url
        return output
    except requests.exceptions.RequestException as e:
        msg = e

In [6]:
def save_dataset(q, urls, year):
    total_count = 0
    for url in urls:
        try:
            data = get_news_data(url)
            if data is None:
                continue
            if q not in os.listdir():
                cwd = os.getcwd()
                os.mkdir(f'{cwd}/{q}')
            total_count += 1
            file_error_symbols = []
            filename = f'{year}-{total_count}'
            f = open(f'./{q}/{filename}.json', "w")
            f.write(json.dumps(data, indent = 4))
            f.close()
        except requests.exceptions.RequestException as e:
            print(e)
    print(f'Saved {total_count} news data in year {year}')
    #print(os.system("npx prettier -w ./dataset/*.json"))

In [7]:
def save_news_urls(q, total, from_year, to_year):
    urls = {}
    for year in range(from_year, to_year + 1):
        urls[str(year)] = get_google_news_urls(q, total, year, year)
        print(f'Year {year} News urls collection - Done')
        #delay request to prevent 429 Too many Request
        if total > 200:
            time.sleep(60)
        else:
            time.sleep(30)
            
    f = open(f'{q}.json', 'w')
    f.write(json.dumps(urls, indent = 4))
    f.close()
    return urls

In [8]:
def get_urls_from_file(filename):
    f = open(filename, "r")
    data = json.load(f)
    f.close()
    return data

In [9]:
def get_keywords_from_file(filename):
    keywords = []
    f = open(filename, "r")
    for keyword in f:
        keywords.append(keyword)
    f.close()
    return list(map(lambda keyword: keyword.strip(), keywords))

In [10]:
def parallel_save_dataset(q, urls):
    start = time.time()
    Parallel(n_jobs=-1)(delayed(save_dataset)(q, urls[year], year) for year in urls)
    end = time.time()
    print('{:.2f} seconds used'.format(end - start))

In [13]:
#run all functions to save news data
def scrap_news_data(total, from_year, to_year, keywords_filename="keywords.txt"):
    #count used time
    start = time.time()
    #step 1 - load keywords file
    keywords = get_keywords_from_file(keywords_filename)
    for query in keywords:
        print(f'Scrapping {query} ...')
        #step 2 - save urls to json file
        save_news_urls(query, total, from_year, to_year)
        #step 3 - get urls object from previous saved json file
        urls = get_urls_from_file(f'{query}.json')
        #step 4 - web scrapping news data from specific year of urls & save into directory seperately
        parallel_save_dataset(query, urls)
        
    end = time.time()
    print(f'[DONE] {end - start} seconds used')

In [367]:
def json_to_csv(q, json_obj, index):
    title = re.sub(r'[,.\'"|]', '', json_obj['title'])
    datetime = json_obj['datetime']
    url = json_obj['url']
    text = re.sub(r'[,.\'"|]', '', json_obj['text'])
    if text != '':
        text = ' '.join(text.encode('utf-8').decode().split())
    if title != '':
        title = ' '.join(title.split()).strip()
    f = open(f'{q}.csv', 'a', encoding='utf-8')
    f.write(f'{index},\"{title}\",{datetime},{url},\"{text}\"\n')
    f.close()

In [368]:
def all_json_to_csv():
    cols = "title,datetime,url,text\n"
    keywords = get_keywords_from_file('keywords.txt')
    for keyword in keywords:
        f = open(f'{keyword}.csv', 'w')
        f.write(cols)
        f.close()
        for index, filename in enumerate(os.listdir(f'./{keyword}')):
            json_file = open(f'./{keyword}/{filename}', 'r')
            json_obj = json.load(json_file)
            json_file.close()
            json_to_csv(keyword, json_obj, index)
    print('Done')

In [369]:
def combine_csv():
    keywords = get_keywords_from_file('keywords.txt')
    dataframes = []
    for keyword in keywords:
        df = pd.read_csv(f'{keyword}.csv', on_bad_lines='skip')
        dataframes.append(df)
    dataframe = pd.concat(dataframes, ignore_index=True)
    dataframe.to_csv('dataset.csv')
    print('Done')

In [353]:
#scrap_news_data(total_news_per_year, from_year, to_year)

In [370]:
all_json_to_csv()

Done


In [371]:
combine_csv()

Done


In [375]:
pd.read_csv('dataset.csv')

,title,datetime,url,text
0,‘KCON All Things Hallyu’ returns to LA,2013-06-27T15:15:35+09:00,https://www.koreaherald.com/view.php?ud=201306...,Most Popular South Korean celebrities grilled ...
1,Choi Ji-woo to play modern-day Mary Poppins,2013-09-16T15:49:54+09:00,https://www.koreaherald.com/view.php?ud=201309...,Most Popular South Korean celebrities grilled ...
2,4 Idola Hallyu Ini Dibilang Paling Mirip Bonek...,2013-12-07T22:00:00+07:00,https://www.liputan6.com/showbiz/read/767787/4...,liputan6 Diperbarui 07 Des 2013 22:00 WIBDiter...
3,JYP Pun Siap Kembali Ramaikan Dunia Hallyu - S...,2013-08-29T13:20:00+07:00,https://www.liputan6.com/showbiz/read/678005/j...,liputan6 Diperbarui 29 Agu 2013 13:20 WIBDiter...
4,Japan honors Jane Birkin for tsunami relief ef...,2013-11-15T15:00:55+09:00,https://www.koreaherald.com/view.php?ud=201311...,Most Popular South Korean celebrities grilled ...
...,...,...,...,...
9101,[CELEB] From liquor to labels Jay Park knows n...,2022-03-09T14:55:44+0900,https://koreajoongangdaily.joins.com/2022/03/0...,Singer rapper and entrepreneur Jay Park [MORE ...
9102,1300: Sydney’s Korean-Australian hip-hop sensa...,2022-05-26T01:30:00+00:00,https://www.smh.com.au/culture/music/korean-au...,We’re sorry this feature is currently unavaila...
9103,What time will Hip Hop Homicides air on WE tv?...,2022-11-02T20:10:24+00:00,https://www.sportskeeda.com/pop-culture/what-t...,Curtis 50 Cent Jacksons highly anticipated eig...
9104,BIBI: Meet The Enigmatic Princess of K-R&B,2022-05-28T13:01:28+00:00,https://rollingstoneindia.com/bibi-meet-the-en...,She grabbed the world’s attention thanks to he...
